## 1. Import libraries.

In [1]:
import numpy as np
import pandas as pd
import pandas as sqrt
from numpy.lib.function_base import average
from numpy import sqrt
import time
from sklearn.metrics import mean_squared_error
import math
import tqdm
from numpy import dot
from numpy.linalg import norm
from sklearn.metrics.pairwise import cosine_similarity
from underthesea import text_normalize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import streamlit as st

import pandas as sqrt
from numpy.lib.function_base import average
from numpy import sqrt
import time
import math
from sklearn.metrics.pairwise import linear_kernel


In [2]:
from keras import losses
from keras.preprocessing.text import Tokenizer
from sklearn.metrics import confusion_matrix,f1_score , accuracy_score
from pyvi import ViTokenizer, ViPosTagger

## 2. Read file data.

In [3]:
df_user = pd.read_csv("USER_DATA_T.csv")
df_job = pd.read_csv("JOB_DATA_T.csv")

In [4]:
df_job['Job Requirements'][0]

'Nam/nữ tốt nghiệp THPT hoặc cao hơn, tuổi không quá 30. Ưu tiên ứng viên có kinh nghiệm CSKH mảng call center/ Telesale, nếu không có kinh nghiệm sẽ được đào tạo miễn phí. Không vướng bận công việc, việc học. Yêu cầu test đánh máy >=35 WPM. Khả năng giao tiếp, giải quyết khiếu nại tốt, giao tiếp thân thiện, có tinh thần cầu tiến. Giọng nói dễ nghe, không quá nặng vùng miền.'

In [5]:
df_user['Skills'][0]

'- Bằng lái xe hạng C, D, E và có kinh nghiệm 5 năm lái xe tải, 5 năm lái xe khách. - Am hiểu về xe tải, biết cách xử lý các vấn đề xảy ra với xe. - Bảo quản và giữ gìn xe sạch sẽ, thường xuyên bảo dưỡng xe. - Lái xe an toàn và đảm bảo tuân thủ luật giao thông. - Khả năng ghi nhớ nhanh chóng, nhớ đường tốt. - Kỹ năng giao tiếp và thiết lập các mối quan hệ với khách hàng, đồng nghiệp. - Kỹ năng giải quyết vấn đề. - Sắp xếp và tổ chức, quản lý công việc hiệu quả, đảm bảo hàng hóa đến tay khách hàng nhanh nhất. - Chịu được áp lực cao trong công việc và khi lái xe.'

## 3. BUIL WORDTOVECTOR.

### Processing data function.

In [4]:
import re

puncts = [ '"', ')', '(', '-', '|',"'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]

def clean_text(x):
    list = []
    for y in x:
        if y not in  puncts:
            list.append(x)
    return list

def tokenbize_word(x):
    x = ViTokenizer.tokenize(str(x))
    return x

def remove_1_length_word(x):
    list = []
    for i in x:
         if len(i)> 1 or i == ',' or i == '.':
            list.append(i)
    return list
# train_df["Comment"] = train_df["Comment"].apply(lambda x: " ".join(remove_1_length_word(x)))
def Processing_data(dataframe):
    dataframe = dataframe.apply(text_normalize)\
        .map(lambda x: tokenbize_word(x))\
            .apply(lambda x: clean_text(x))\
                .apply(lambda x : x[0])\
                    #.replace(r'\W+', ' ', regex=True)
    return dataframe

In [5]:
# Processing 'job Requirements' columns

df_job['Job Requirements'] = Processing_data(df_job['Job Requirements'])

In [6]:
df_user['Skills'] = Processing_data(df_user['Skills'])

In [7]:
df_user['Skills'][0]

'- Bằng lái_xe hạng C , D , E và có kinh_nghiệm 5 năm lái xe_tải , 5 năm lái xe_khách . - Am_hiểu về xe_tải , biết cách xử_lý các vấn_đề xảy ra với xe . - Bảo_quản và giữ_gìn xe sạch_sẽ , thường_xuyên bảo_dưỡng xe . - Lái_xe an_toàn và đảm_bảo tuân_thủ luật giao_thông . - Khả_năng ghi_nhớ nhanh_chóng , nhớ đường tốt . - Kỹ_năng giao_tiếp và thiết_lập các mối quan_hệ với khách_hàng , đồng_nghiệp . - Kỹ_năng giải_quyết vấn_đề . - Sắp_xếp và tổ_chức , quản_lý công_việc hiệu_quả , đảm_bảo hàng_hóa đến tay khách_hàng nhanh nhất . - Chịu được áp_lực cao trong công_việc và khi lái_xe .'

### Create WordEmbedding matrix.

In [7]:
X_train=df_user['Skills'].append(df_job['Job Requirements'])

In [8]:
word_dict = []
embeddings_index = {}
embedding_dim = 100
max_feature = len(embeddings_index) + 2

f = open('word2vec_vi_words_100dims.txt', encoding="utf8")
for line in f:
    values = line.split(' ')
    word = values[0] 
    word_dict.append(word)
    try:
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    except Exception as e:
        pass
f.close()

print('Embedding data loaded')

Embedding data loaded


In [9]:
max_len = 50

word_tokenizer = Tokenizer(oov_token=-1)
word_tokenizer.fit_on_texts(X_train.values)
word_to_index = word_tokenizer.word_index
word_to_index['pad'] = 0
word_to_index['unk'] = -1

index_to_word = {i: w for w, i in word_to_index.items()}

def encoding(X):
    sentences = []
    
    for t in X:
        tokenized_sentence = ViTokenizer.tokenize(t)
        sentences.append(tokenized_sentence)
    
    X = word_tokenizer.texts_to_sequences(sentences)
    X = pad_sequences(maxlen = max_len, sequences = X, padding = "post", value = word_to_index['pad'])

    return X

In [10]:
num_words=len(word_to_index)

In [11]:
import numpy as np
# first create a matrix of zeros, this is our embedding matrix
embedding_matrix = np.zeros((num_words, embedding_dim))

# for each word in out tokenizer lets try to find that work in our w2v model
for word,i in word_to_index.items():
    if  i> max_feature:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
    else:
        # doesn't exist, assign a random vector
        embedding_matrix[i] = np.random.randn(embedding_dim)

## 4. Recommend Fuction.

In [12]:
df_user.columns

Index(['URL User', 'UserID', 'User Name', 'Industry', 'Desired Job',
       'Workplace Desired', 'Desired Salary', 'Gender', 'Marriage', 'Age',
       'Target', 'Skills', 'Degree', 'Work Experience'],
      dtype='object')

In [13]:
overview_matrix1 = encoding(df_user['Skills'])

In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# Xử lý
job = df_job['Industry'].to_list()
user = df_user['Industry'].to_list()
corpus_merge  = job + user
corpus_merge

In [16]:
vectorizers = CountVectorizer()
Y = vectorizers.fit_transform(corpus_merge)
Vocabulary = vectorizers.get_feature_names_out()
# TF- IDF
vectorizer = TfidfVectorizer(max_features= 617,vocabulary= Vocabulary)

overview_matrix1 = vectorizer.fit_transform(df_user['Industry'])


In [17]:

def get_recommendation(userid, location, num_jobs):
  

  # Create new job dataframe by location
  df_nex_job =  df_job[df_job['Job Address']==location].reset_index(drop=True)
  df_nex_job['JobID']=[x for x in range(len(df_nex_job))]
  #job_matrix =  encoding(df_nex_job['Job Requirements'])
  job_matrix= vectorizer.fit_transform(df_nex_job['Industry'])
  
  #Caculate matrix.
  cosine_sim = linear_kernel(overview_matrix1, job_matrix)


  # get index by user id 
  v = df_user[df_user['UserID'] == userid ].index.values.astype(int)[0]
  # Sắp xép dựa trên điểm số tương tự
  sim_scores = list(enumerate(cosine_sim[v]))

  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

  sim_scores = sim_scores[0:num_jobs]

  job_indices = [i[0] for i in sim_scores]

  df = df_nex_job[df_nex_job['JobID'].isin(job_indices) ].reindex(job_indices)

  return df



In [18]:
df_1 = get_recommendation(986657,"Hồ Chí Minh",50)
df_2 = get_recommendation(966890,"Hồ Chí Minh",50)
df_3 = get_recommendation(990765,"Hồ Chí Minh",50)

In [19]:
df_1.to_csv("Result_W2V_986657_Industry.csv",index = False )

In [20]:
df_2.to_csv("Result_W2V_966890_Industry.csv",index = False )

In [21]:
df_3.to_csv("Result_W2V_990765_Industry.csv",index = False )

In [22]:
df_1

,URL Job,Job Title,Name Company,Company Overview,Company Size,Company Address,Job Description,Job Requirements,Benefits,Job Address,Job Type,Gender,Number Cadidate,Career Level,Years of Experience,Salary,Submission Deadline,Industry,JobID
414,https://timviec365.vn/nhan-vien-it-p841287.html,NHÂN VIÊN IT,CÔNG TY ĐỊA ỐC MINH ĐỨC,"281D Lãnh Binh Thăng, Phường 8, Quận 11, Tp Hồ...",100-499,"281D Lãnh Binh Thăng, Phường 8, Quận 11, Tp Hồ...","- Giám sát công tác hỗ trợ người dùng, sửa chữ...",- Có kiến_thức cơ_bản về hệ_thống mạng - Có ít...,"- Mức lương: 10,5 triệu/tháng + hoa hồng + Thư...",Hồ Chí Minh,Full time,Không yêu cầu,2,Thực tập sinh,Dưới 1 năm,"15,000,000 - 20,000,000",15/12/2022,it phần mềm,414
744,https://timviec365.vn/designer-nhan-vien-thiet...,[Designer] Nhân Viên Thiết Kế,công ty cổ phần care with love,Đang cập nhật,Đang cập nhật,102s An Dương Vương phường 9 quận 5,"- Lên ý tưởng, & triển khai các phương án thiế...",- Tốt_nghiệp Cao_đẳng / Đại_học trở lên các ng...,- Được làm việc trong một công ty hàng đầu về ...,Hồ Chí Minh,Full time,Không yêu cầu,1,Nhân viên,1-3 năm,"5,000,000 - 10,000,000",31/12/2022,it phần mềm,744
1000,https://timviec365.vn/phudonggroup-chuyen-vien...,PHUDONGGROUP – CHUYÊN VIÊN IT,CÔNG TY CỔ PHẦN ĐỊA ỐC PHÚ ĐÔNG**,Chúng tôi luôn tạo mọi điều kiện để CBNV phát ...,100-499,"55 Trần Nhật Duật, Phường Tân Định, Quận 1.","- Làm việc tại trụ sở Quận 1, địa chỉ: 55 Trần...",- Tốt_nghiệp cao_đẳng / đại_học chuyên_ngành c...,- Thu nhập : 10tr đến 15tr - Hưởng các chế độ ...,Hồ Chí Minh,Full time,Không yêu cầu,1,Nhân viên,1-3 năm,"10,000,000 - 15,000,000",15/12/2022,it phần mềm,1000
1689,https://timviec365.vn/cong-ty-tnhh-tu-van-thuo...,Công Ty Tnhh Tư Vấn Thương Mại Khánh Linh Tuyể...,công ty TNHH tư vấn thương mại khánh linh,LĨNH VỰC HOẠT ĐỘNG 1. Tư vấn thành lập doanh n...,500-999,"Phòng 1702, tòa nhà 17T4 Hoàng Đạo Thúy, Thanh...",Lập trình phát triển các dự án của công ty: ph...,Bằng_cấp ( tối_thiểu ) : Cử_nhân CNTT hoặc tươ...,: - Lương/thưởng: thỏa thuận Các chế độ đãi ng...,Hồ Chí Minh,Full time,Không yêu cầu,10,Nhân viên,1-3 năm,"5,000,000 - 10,000,000",30/05/2024,it phần mềm,1689
1696,https://timviec365.vn/tuyen-dung-lap-trinh-vie...,Tuyển dụng Lập trình viên IT,Công Ty Cổ Phần Đầu Tư Thương Mại Dịch Vụ D&D,Công ty cổ phần đầu tư thương mại dịch vụ D&D ...,100-499,"4449 Nguyễn Cửu Phú, Tân Tạo A, Bình Tân","Thiết kế web, app Chạy marketing online",Đang cập_nhật,"Thưởng theo dự án, tiến độ",Hồ Chí Minh,Full time,Nam,5,Nhân viên,1-3 năm,"10,000,000 - 15,000,000",31/12/2022,it phần mềm,1696
949,https://timviec365.vn/nhan-vien-kinh-doanh-pha...,NHÂN VIÊN KINH DOANH PHẦN MỀM QUẢN LÝ BÁN HÀNG,Công Ty CP Công Nghệ Sapo 01,Sapo là công ty Công nghệ. Với khả năng và hướ...,Đang cập nhật,Tầng 5 70 Lữ Gia Phường 15 Quận 11,"- Tìm kiếm, tư vấn, giới thiệu cho Khách hàng ...","- Là người giao_tiếp tốt , cầu thị , ham học_h...",- Thu nhập hấp dẫn lên đến 20 triệu/tháng: Lươ...,Hồ Chí Minh,Full time,Không yêu cầu,10,Nhân viên,Không yêu cầu kinh nghiệm,"10,000,000 - 15,000,000",07/01/2023,nhân viên kinh doanh phần mềm,949
980,https://timviec365.vn/nhan-vien-kinh-doanh-qua...,NHÂN VIÊN KINH DOANH QUẬN 11,Công Ty CP Công Nghệ Sapo 01,Sapo là công ty Công nghệ. Với khả năng và hướ...,Đang cập nhật,Tầng 5 70 Lữ Gia Phường 15 Quận 11,"- Tìm kiếm, tư vấn, giới thiệu cho Khách hàng ...","- Là người giao_tiếp tốt , cầu thị , ham học_h...",- Thu nhập không giới hạn: Lương cơ bản từ 6 t...,Hồ Chí Minh,Full time,Không yêu cầu,10,Nhân viên,Không yêu cầu kinh nghiệm,"10,000,000 - 15,000,000",06/01/2023,nhân viên kinh doanh phần mềm,980
1627,https://timviec365.vn/q12-tuyen-tts-kinh-doanh...,Q12 tuyển TTS kinh doanh hỗ trợ thu nhập,Công Ty Tnhh Tm & Dv Nina Vietnam,"Công ty TNHH TM DV NINA (Nina Co.,Ltd.) được c...",Đang cập nhật,"Tòa Nhà Saigon Tel , Lô 46 - Cvpm Quang Trung,...","— Tìm kiếm thông tin, tiếp cận khách hàng tiềm...","— Có khả_năng giao_tiếp , làm_việc độc_lập và ...",— Hỗ trợ 3-5tr/ tháng— Có cơ hội trở thàn